In [32]:
import panel as pn
pn.extension('katex')
import param
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp

import hvplot.pandas
import holoviews as hv
from bokeh.models.formatters import PrintfTickFormatter, FuncTickFormatter
import cmocean

from ode_solver import curvature_evolution, ODEsol, reefcrest

In [35]:
class Explore(param.Parameterized):
    title = '## Streamwise Vorticity IVP'
    
    distance = param.Number(default = 10)
    radius = param.Number(default = 12)
    slope = param.Number(default = -2)
    drag = param.Number(default = -2.35)
    latitude = param.Number(default = -30)
    
    initial_u = param.Number(default = 0.25)
    initial_h = param.Number(default = 20)
    initial_t = param.Number(default = 0)
    initial_a = param.Number(default = 0)
    
    columns = ["s","r","𝜃","x","y","h","u","k","angle","dhds","dhdn"]
    size = param.Selector(objects=columns, default = "u") 
    color = param.Selector(objects=columns, default = "k") 
    #centering = param.Selector(objects=['jet','island'], default = "island")
    #alpha = param.Selector(objects=columns, default = "s")
    
    select_x = param.Selector(objects=columns, default = "s") 
    select_y = param.Selector(objects=columns, default = "k")
    
    @pn.depends('radius')
    def ellipse(self):
        return hv.Ellipse(0,0,2*self.radius).opts(line_dash = "dashed")
    
    def plot_xy(self,ds, formatter = '%.1e'):
        """scatter plot w/ variables affecting marker size and color"""
        
        clim = (ds.color.min(), ds.color.max())
        
#         if  max(abs(np.log(np.array(clim)))) > 3:
#             formatter = '%.1e'
#         else:
#             formatter = '%.2f'
        
        cff = self.ellipse()*ds.hvplot.scatter(x = "x", y = "y", c = "color", s = "size").opts(clim = clim,
                            colorbar_opts={'formatter': PrintfTickFormatter(format=formatter), 'padding': 0})
        
#        if ds.centering == 'jet':
#             xlim = (ds.x.values.min(), ds.x.values.max())
#             ylim = (ds.y.values.min(), ds.y.values.max())
#         elif ds.centering == 'island':
        rmax = ds.r.values.max()
        xlim = (-rmax, rmax)
        ylim = (-rmax, rmax)
            
        return cff.opts(xlim = xlim , ylim = ylim)
    
    def plot_dia(self,df): 
        """alongpath curvature equation diagnostics by term"""
        
        curves = df.hvplot(x = "s", y = ["nonlinear","coriolis","spreading","slope_torque","dissipation","dkds_diagnostic"],
                                    color = ["r","b","g","m","orange","black"],
                                    line_dash = ["solid","solid","solid","solid","solid"])
        values = df[ ["nonlinear","coriolis","spreading","slope_torque","dissipation","dkds_diagnostic"] ]
        xlim = (0,df.s.max())
        ylim = values.min().min(),values.max().max()
        
        return curves.opts(xlabel = "streamwise distance (km)", ylabel = "m⁻²", xlim = xlim, ylim = ylim, 
                           legend_position = "right", yformatter='%.1e')
    
    @pn.depends('distance','radius','slope','drag','latitude','initial_u','initial_h','initial_t','initial_a','size','color','select_x','select_y')
    def solve_ode(self):
        """ODE solver"""
        slope = 10**(self.slope)
        f = 2*7.29e-5*np.sin(self.latitude*np.pi/180)
        drag = 10**self.drag
        k0 = -f/self.initial_u
        y0 = self.initial_a, self.radius*1e3, self.initial_t, self.initial_h, self.initial_u, k0
        Q = ODEsol(slope , f,  drag, ri = self.radius, dadn = 0)
        sol = solve_ivp(curvature_evolution, t_span = [Q.start, self.distance*1e3], y0 = y0, args = (slope, f, drag, self.radius*1e3, 0), 
                        method = "LSODA", dense_output = True, events = [reefcrest])
        s = np.linspace(Q.start, sol.t[-1], 100).flatten()
        ds = Q.state(sol, s)
        df = Q.diagnostics()

        ds["color"] = ds[self.color]
        ds["size"] = 100*( (ds[self.size].values - ds[self.size].min())/np.ptp(ds[self.size]) + 0.1)

        plot_xy = self.plot_xy(ds).opts(
                                xlabel = "x (km)", ylabel = "y (km)", data_aspect = 1, frame_height = 500, frame_width = 500, 
                                fontsize={'title': 16, 'labels': 16, 'ticks': 12})
        plot_dia = self.plot_dia(df).opts(frame_height = 205, fontsize={'title': 16, 'labels': 16, 'ticks': 12})
        plot_phase = ds.hvplot.line(x = self.select_x, y = self.select_y).opts(frame_height= 205, frame_width = 425, color = "black", fontsize={'title': 16, 'labels': 16, 'ticks': 12})

        return plot_xy + plot_dia + plot_phase
                 
               
    def viewable(self):
        
        widgets = {}
        widgets["distance"] = pn.Param(self.param.distance, widgets = {"distance": pn.widgets.FloatSlider(name = "Integration Distance", start = 1, end = 100, step = 1, value = 10, format=PrintfTickFormatter(format='%d km'))} )
        widgets["radius"] = pn.Param(self.param.radius,  widgets = {"radius": pn.widgets.FloatSlider(name = "Radius of Island", start = 1, end = 100, step = 1, value = 12, format=PrintfTickFormatter(format='%d km'))} )
        widgets["slope"] = pn.Param(self.param.slope,  widgets = {"slope": pn.widgets.FloatSlider(name = "Slope (𝛽)", start = -3, end = 0, step = .05, value = -2, format=FuncTickFormatter(code="""return (10**tick).toPrecision(3)"""))} )
        widgets["drag"] = pn.Param(self.param.drag,  widgets = {"drag": pn.widgets.FloatSlider(name = "Drag Coefficient (Cd)", start = -3, end = 0, step = .05, value = -2.35, format=FuncTickFormatter(code="""return (10**tick).toPrecision(3)"""))} )
        widgets["latitude"] = pn.Param(self.param.latitude,  widgets = {"latitude":  pn.widgets.FloatSlider(name = "latitude", start = -90, end = 90, step = 1, value = -30, format=PrintfTickFormatter(format='%.1f°'))} )

        widgets["u"] = pn.Param(self.param.initial_u, widgets = { "initial_u": pn.widgets.FloatSlider(name = "Initial Speed", start = 0.01, end = 1, step = 0.01, value = 0.25, format=PrintfTickFormatter(format='%0.2f (m/s)'))} )
        widgets["h"] = pn.Param(self.param.initial_h,  widgets = {"initial_h": pn.widgets.FloatSlider(name = "Initial Depth", start = 1, end = 100, step = 1, value = 20, format=PrintfTickFormatter(format='%d (m)'))} )
        widgets["t"]= pn.Param(self.param.initial_t,  widgets = {"initial_t": pn.widgets.FloatSlider(name = "Initial Azimuth", start = 0, end = 2*np.pi, step = 2*np.pi/180, value = 0, format=FuncTickFormatter(code="""return (tick*180/3.14).toPrecision(3) + '\u00B0' """))} )
        widgets["a"] = pn.Param(self.param.initial_a,  widgets = {"initial_a": pn.widgets.FloatSlider(name = "Initial Heading", start = 0, end = 2*np.pi, step = 2*np.pi/180, value = 0, format=FuncTickFormatter(code="""return (tick*180/3.14).toPrecision(3) + '\u00B0' """))} )

        widgets["select_x"] = pn.Param(self.param.select_x, widgets = {"select_x": pn.widgets.Select(name='x-axis', value = "s", options=self.columns)})
        widgets["select_y"] = pn.Param(self.param.select_y, widgets = {"select_y": pn.widgets.Select(name='y-axis', value = "r", options=self.columns)})

        widgets["size"] = pn.Param(self.param.size, widgets = { "size":  pn.widgets.Select(name='scatter size', value = "u", options=self.columns)})
        widgets["color"] = pn.Param(self.param.color, widgets = {"color": pn.widgets.Select(name='scatter color', value = "k", options=self.columns)})

        #widgets["centering"] = pn.Param(self.param.centering, widgets = {"centering": pn.widgets.Select(name='centering', options=['jet','island'], size = 2)})

        widgets["parameters"] = pn.WidgetBox('### IVP Parameters', widgets["distance"], widgets["radius"],  widgets["slope"], widgets["drag"], widgets["latitude"])
        widgets["initial conditions"] = pn.WidgetBox("### IVP Initial Conditions", widgets["u"], widgets["h"], widgets["t"], widgets["a"])
        widgets["scatter options"] = pn.WidgetBox("### Scatter options", widgets["size"], widgets["color"])#, widgets["centering"])
        widgets["phase space"] = pn.WidgetBox("### Line Options", widgets["select_x"], widgets["select_y"])
    
        p = hv.DynamicMap(self.solve_ode)
        
        return pn.Column(self.title,pn.Row( pn.Column(widgets["parameters"],  widgets["initial conditions"]), pn.Column(p.collate()[0],widgets["scatter options"]), pn.Column( p.collate()[1],p.collate()[2], widgets["phase space"]) ))

dashboard = Explore()
dashboard.viewable().servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] WidgetBox
                [0] Markdown(str)
                [1] Param(Explore, parameters=['distance'], show_name=False, widgets={'distance': FloatSlider(e...})
                [2] Param(Explore, parameters=['radius'], show_name=False, widgets={'radius': FloatSlider(end...})
                [3] Param(Explore, parameters=['slope'], show_name=False, widgets={'slope': FloatSlider(end=...})
                [4] Param(Explore, parameters=['drag'], show_name=False, widgets={'drag': FloatSlider(end=0...})
                [5] Param(Explore, parameters=['latitude'], show_name=False, widgets={'latitude': FloatSlider(e...})
            [1] WidgetBox
                [0] Markdown(str)
                [1] Param(Explore, parameters=['initial_u'], show_name=False, widgets={'initial_u': FloatSlider(...})
                [2] Param(Explore, parameters=['initial_h'], show_name=False, widgets={'initial_h': FloatSlider(...})
                [3] Param(Explore, parameters=['initial_t'], show_name=False, widgets={'initial_t': FloatSlider(...})
                [4] Param(Explore, parameters=['initial_a'], show_name=False, widgets={'initial_a': FloatSlider(...})
        [1] Column
            [0] HoloViews(DynamicMap)
            [1] WidgetBox
                [0] Markdown(str)
                [1] Param(Explore, parameters=['size'], show_name=False, widgets={'size': Select(name='scat...})
                [2] Param(Explore, parameters=['color'], show_name=False, widgets={'color': Select(name='sca...})
        [2] Column
            [0] HoloViews(DynamicMap)
            [1] HoloViews(DynamicMap)
            [2] WidgetBox
                [0] Markdown(str)
                [1] Param(Explore, parameters=['select_x'], show_name=False, widgets={'select_x': Select(name='...})
                [2] Param(Explore, parameters=['select_y'], show_name=False, widgets={'select_y': Select(name='...})